In [2]:
# This is the package we will be using for interacting with xml
!pip install lxml
from lxml import etree

'pip' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
# All XML objects have a root element
root = etree.Element("root")

# Lets define a function that makes it a bit easier to visualize the tree
def print_tree(root):
    print(etree.tostring(root, pretty_print=True))

# We can use the function like this:
print_tree(root)

b'<root/>\n'


In [7]:
# We can add elements to the root by using SubElement
etree.SubElement(root, "child1")
etree.SubElement(root, "child2")
etree.SubElement(root, "child3")

print_tree(root)

b'<root>\n  <child1/>\n  <child2/>\n  <child3/>\n</root>\n'


In [9]:
# We can iterate over all the children of a node by using a for loop

for child in root:
    print(child) # This will print the object, i.e <Element child1 at 0x1d5c17f0080
    print(child.tag) # This will print the child's name (for instance "child1")

    

<Element child1 at 0x1d5c1820180>
child1
<Element child2 at 0x1d5c1822480>
child2
<Element child3 at 0x1d5c18237c0>
child3


In [46]:
# We can also use the lxml package to read xml files into a tree object
# Download this file https://www.w3schools.com/xml/note.xml and place it in the same folder as this notebook 

# First we have to open the file and get the xml as a string
def xml_from_file(filename):
    with open(filename, "r", encoding="utf-8") as f:
        xmlstring = f.read()
        return etree.fromstring(xmlstring.encode())

# Lets try to see if we can read a simple xml string
root = xml_from_file("note.xml")
# We can see that we are able to print its children
for child in root:
    print(child)


<Element to at 0x1d5bf5b5d80>
<Element from at 0x1d5c1912000>
<Element heading at 0x1d5c1906280>
<Element body at 0x1d5bf5b5d80>


In [47]:
# We can also access the elements using its tags:
for child in root:
    print(child.tag, child.text)

to Tove
from Jani
heading Reminder
body Don't forget me this weekend!


In [48]:
# If you want a specific field for all children, you can use iter
for child in root.iter("to"):
    print(child.tag, child.text)

to Tove


In [49]:
# ... but this was just a reaally small xml file
# Lets try with a bigger one: https://www.w3schools.com/xml/plant_catalog.xml 
# We can reuse the functions we created above
root = xml_from_file("plant_catalog.xml")
for child in root[0:2]: # Print the first two children
    print(child.tag, child.getchildren())



PLANT [<Element COMMON at 0x1d5c1941a80>, <Element BOTANICAL at 0x1d5c19ac480>, <Element ZONE at 0x1d5c1abf000>, <Element LIGHT at 0x1d5c1abce40>, <Element PRICE at 0x1d5c1abd840>, <Element AVAILABILITY at 0x1d5c1aae240>]
PLANT [<Element COMMON at 0x1d5c192ce40>, <Element BOTANICAL at 0x1d5c1abdd40>, <Element ZONE at 0x1d5c1abf000>, <Element LIGHT at 0x1d5c1abce40>, <Element PRICE at 0x1d5c1abd840>, <Element AVAILABILITY at 0x1d5c1aacd80>]


In [52]:
# Lets get the common names of all plants:
for child in root.iter("COMMON"):
    print(child.text)

Bloodroot
Columbine
Marsh Marigold
Cowslip
Dutchman's-Breeches
Ginger, Wild
Hepatica
Liverleaf
Jack-In-The-Pulpit
Mayapple
Phlox, Woodland
Phlox, Blue
Spring-Beauty
Trillium
Wake Robin
Violet, Dog-Tooth
Trout Lily
Adder's-Tongue
Anemone
Grecian Windflower
Bee Balm
Bergamot
Black-Eyed Susan
Buttercup
Crowfoot
Butterfly Weed
Cinquefoil
Primrose
Gentian
Blue Gentian
Jacob's Ladder
Greek Valerian
California Poppy
Shooting Star
Snakeroot
Cardinal Flower


In [42]:
# From the output above we can see that this xml is deeper - it has children that has children!
# Lets try to print a single child:

for child in root[0]: #Select only the first child of root and print its children
    print(child.tag, child.text)


COMMON Bloodroot
BOTANICAL Sanguinaria canadensis
ZONE 4
LIGHT Mostly Shady
PRICE $2.44
AVAILABILITY 031599


In [43]:
# Now that we know what a single plant looks like, we can try to loop through every child and map it to a dict
# We will end up with a list of dicts, which is perfect for our csv.DictWriter! 

rows = []
for plant in root:
    plantdict = {} # Create a new plant dict for each row
    for attributes in plant:
        # print(attributes.tag, attributes.text)
        plantdict[attributes.tag] = attributes.text
        rows.append(plantdict)
print(rows[0:3])# Print some of the rows as a demo


[{'COMMON': 'Bloodroot', 'BOTANICAL': 'Sanguinaria canadensis', 'ZONE': '4', 'LIGHT': 'Mostly Shady', 'PRICE': '$2.44', 'AVAILABILITY': '031599'}, {'COMMON': 'Bloodroot', 'BOTANICAL': 'Sanguinaria canadensis', 'ZONE': '4', 'LIGHT': 'Mostly Shady', 'PRICE': '$2.44', 'AVAILABILITY': '031599'}, {'COMMON': 'Bloodroot', 'BOTANICAL': 'Sanguinaria canadensis', 'ZONE': '4', 'LIGHT': 'Mostly Shady', 'PRICE': '$2.44', 'AVAILABILITY': '031599'}]


In [36]:
# Now that we have a list of dicts we can use it to write to a csv

import csv 

with open("plants.csv", "w") as f:
    writer = csv.DictWriter(f, fieldnames=["COMMON", "BOTANICAL", "PRICE", "LIGHT", "AVAILABILITY", "ZONE"])
    writer.writeheader()
    writer.writerows(rows)

## Exercise

1. Load [this](https://www.w3schools.com/xml/simple.xml) CSV and play around with it. Try to print some of the elements to get a feel of how this XML looks.
2. Iterate over all the children of the root node and print the Food name of each menu item. 
3. Create a function that accepts input from the user (using input()) and finds the Food item that has the same name as the one the user specified. E.g if the user writes "French Toast", the function should return 
```
name: French Toast
price: $4.50
description: Thick slices made from our homemade sourdough bread
calories: 600
```
If there is no item with the same name, it should return
```
Sorry we don't have that food!
```
4. Iterate through all the items to find the item with the highest price.
Hint: `"$4.50".strip('$')` removes the dollar part of the string, so that you can convert it to float for comparison
5. Write all the Food that costs over $6 on the menu to a CSV with columns `name, price, description,calories`

In [3]:
def xml_from_file(filename):
    with open(filename, "r", encoding="utf-8") as f:
        xmlstring = f.read()
        return etree.fromstring(xmlstring.encode())

file = xml_from_file('breakfast_menu.xml')

In [29]:
for child in file[0]:
    print(child.tag, child.text)

name Belgian Waffles
price $5.95
description Two of our famous Belgian Waffles with plenty of real maple syrup
calories 650


In [35]:
def print_names(file):
    for child in file.iter('name'):
        print(child.text)

print_names(file)

Belgian Waffles
Strawberry Belgian Waffles
Berry-Berry Belgian Waffles
French Toast
Homestyle Breakfast


In [65]:
user_input = input('Order: ').strip().title()

def child_tags(file, i):
    for child in file[i]:
        print(child.tag, child.text)

def get_food(file, user_input):
    for count, child in enumerate(file.iter('name')):
        if user_input == child.text:
            child_tags(file, count)
        else:
            continue

name French Toast
price $4.50
description Thick slices made from our homemade sourdough bread
calories 600
Sorry, we don't have that food


In [7]:
def get_food_names(file):
    food_names = []
    for child in file.iter('name'):
        food_names.append(child.text)
    return food_names

def test_input(user_input, food_names):
    if user_input not in food_names:
        return True

def print_food(file, user_input):
    index = food_list.index(user_input)
    for child in file[index]:
        print(child.tag, child.text)


user_input = input('Choice: ').strip().title()
food_list = get_food_names(file)
if test_input(user_input, food_list):
    print("Sorry, we don't have that food")
else:
    print_food(file, user_input)




name Strawberry Belgian Waffles
price $7.95
description Light Belgian waffles covered with strawberries and whipped cream
calories 900
